
# 📘 Ollama 使用说明及无管理员权限时的应对方案

Ollama 是一个方便的本地大模型推理工具，但它的安装通常需要管理员权限（特别是在 Windows 和 macOS 下）。  
如果没有管理员权限，直接安装 Ollama 可能会遇到权限不足的问题。

## 🔹 没有管理员权限时的应对方式

1. **使用便携版 Ollama**
   - 从 GitHub Releases 页面下载可执行文件，放在用户目录下运行，不写入系统目录。
   - 启动后同样可以通过 `http://localhost:11434` 调用 API。

2. **使用 llama.cpp / llama-cpp-python 替代**
   - 在用户目录下安装：  
     ```bash
     pip install --user llama-cpp-python
     ```  
   - 不需要管理员权限，即可本地加载 `.gguf` 格式模型运行。

3. **远程调用 Ollama 服务**
   - 在有权限的服务器或另一台电脑上安装 Ollama。
   - 在本机程序中通过 HTTP 请求调用远程的 Ollama 接口，例如：  
     ```python
     import requests

     res = requests.post("http://server-ip:11434/api/generate", json={
         "model": "mistral",
         "prompt": "Hello!"
     })
     print(res.json())
     ```

4. **使用云端 API 替代**
   - 如果只需要调用模型，可以使用 Hugging Face Hub 或 OpenAI API。
   - 不依赖本地安装，避免权限问题。

## ✅ 推荐做法
- **优先选择 llama-cpp-python**，适合没有管理员权限的本地环境。
- 如果硬件不够或依赖安装受限，可以选择 **远程调用** 或 **云 API**。


RAG 系统入门指南 - Python 实现

所需库和工具清单

In [ ]:
pip install --user llama-cpp-python

In [ ]:
pip install -U langchain-community

In [ ]:
pip install -U langchain-huggingface

In [ ]:
pip install hf_xet

In [ ]:
pip install -U langchain-openai

In [1]:
pip install -U langchain-ollama

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement lanngchain_ollama (from versions: none)
ERROR: No matching distribution found for lanngchain_ollama


系统架构解析
RAG系统主要包含三个组件：

检索器：从知识库中找到与用户查询最相关的文本片段
嵌入引擎：将文档内容转化为向量以便高效检索
生成器：基于检索到的内容创建最终回答
下面是完整的代码实现：

In [2]:
import os
from langchain_ollama import OllamaLLM
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from PyPDF2 import PdfReader  # 处理PDF文件
from docx import Document  # 处理Word文档

C:\Users\81803\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def load_docx(file_path):
    """加载Word文档内容"""
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

In [ ]:
def load_pdf(file_path):
    """加载PDF文件内容"""
    text = ""
    with open(file_path, 'rb') as f:
        pdf_reader = PdfReader(f)
        for page in pdf_reader.pages:
            text += page.extract_text() or ""  # 处理可能的None值
    return text

In [4]:
# ===============================
# 文件加载函数
# ===============================
def load_file(file_path, file_type):
    """根据文件类型加载不同文档"""
    if file_type == 'pdf':
        return load_pdf(file_path)
    elif file_type == 'docx':
        return load_docx(file_path)
    else:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()

# ===============================
# 初始化本地 Ollama LLM
# ===============================
def init_llm(model_name="deepseek-r1:8b"):
    """初始化本地 Ollama 模型"""
    return OllamaLLM(
        model=model_name,
        temperature=0
    )
# ===============================
# RAG 主流程
# ===============================
def rag_pipeline(documents_dir, user_query, model_name="deepseek-r1:8b"):
    """RAG 流程：加载文档 → 向量索引 → 问答"""
    # 1. 加载文档
    all_texts = []
    for file_name in os.listdir(documents_dir):
        file_path = os.path.join(documents_dir, file_name)
        if os.path.isfile(file_path) and file_name.lower().endswith(('.pdf', '.docx', '.txt')):
            ext = file_name.split('.')[-1].lower()
            text = load_file(file_path, ext)
            all_texts.append({'content': text, 'filename': file_name})

    if not all_texts:
        return "❌ 没有找到任何文档，请检查目录路径"

    print("✅ 文档已加载，创建向量索引...")

    # 2. 嵌入模型（这里用 HuggingFace sentence-transformer，本地可跑）
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

    # 3. 创建向量数据库
    metadatas = [{"source": doc['filename']} for doc in all_texts]
    db = FAISS.from_texts(
        [doc['content'] for doc in all_texts],
        embeddings,
        metadatas=metadatas
    )
    db.save_local("faiss_index")
    print("✅ 向量数据库已创建")

    # 4. 初始化本地 LLM
    llm = init_llm(model_name=model_name)

    # 5. 创建检索器
    retriever = db.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 5}
    )

    # 6. 创建问答链（新版写法）
    prompt_template = "根据以下文档回答问题：\n{context}\n\n问题: {question}\n答案:"
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=True
    )

    print("✅ 问答链已准备好，正在回答问题...")

    try:
        response = qa_chain.invoke({"query": user_query})

        # 提取答案和来源
        answer = response["result"]
        source_docs = response.get("source_documents", [])

        sources = []
        for doc in source_docs:
            metadata = doc.metadata
            page_content = doc.page_content
            if isinstance(metadata, dict) and "source" in metadata:
                snippet = page_content[:50] + "..." if len(page_content) > 50 else page_content
                sources.append({"filename": metadata["source"], "text": snippet})

        return {"answer": answer, "sources": sources}

    except Exception as e:
        print(f"❌ 处理查询时出错: {e}")
        return "抱歉，我无法回答这个问题，请提供更多上下文信息。"

In [5]:
# ===============================
# 主程序入口
# ===============================
if __name__ == "__main__":
    documents_dir = "./my_documents"
    query = input("请输入你的问题: ")

    # 默认使用本地 deepseek-r1:8b 模型
    result = rag_pipeline(documents_dir, query, model_name="deepseek-r1:8b")

    if isinstance(result, dict):
        print("\n📌 答案:")
        print(result["answer"])

        print("\n📌 相关内容来源（片段）:")
        for src in result["sources"]:
            print(f"- 来源: {src['filename']}")
            print(f"  内容: {src['text']}\n")
    else:
        print(result)


✅ 文档已加载，创建向量索引...
✅ 向量数据库已创建
✅ 问答链已准备好，正在回答问题...

📌 答案:
<think>
嗯，用户分享了一个关于使用LlamaIndex构建中医临床诊疗智能助手的PDF文档，并询问从中学到什么。首先需要理解这个文档的内容和结构。

好的，我看到了很多代码片段，看起来是关于如何使用LlamaIndex结合本地大模型（如Qwen-7B）来实现检索增强生成(RAG)系统的教程或笔记。这些内容涵盖了多个方面：

1. **基础RAG流程**：介绍了如何构建一个基于文档的问答系统。
2. **本地部署模型**：展示了如何在LlamaIndex中使用HuggingFaceLLM和HuggingFaceEmbedding来加载本地的大语言模型（如Qwen）和嵌入模型（如bge-base-zh-v1___5）。
3. **自定义Prompt模板**：提供了如何自定义英文prompt template的方法，并展示了如何用中文编写这些模板。

用户还提到了一个查询示例："不耐疲劳，口燥、咽干可能是哪些证候？"，并展示了一个完整的RAG流程。这包括检索相关文档片段，然后根据这些片段生成回答。

现在，我需要思考一下从这个PDF中能学到什么内容：

- **如何使用LlamaIndex进行RAG**：了解了如何构建一个基于检索增强的问答系统。
- **如何自定义Prompt模板**：学会了如何在LlamaIndex中使用中文prompt template，并更新到查询引擎中。
- **模型和嵌入模型的选择与配置**：知道如何选择并加载本地的大语言模型（LLM）和嵌入模型，以及它们在RAG中的作用。

用户的问题是：“中医临床诊疗智能助手.pdf中能学到什么”，我需要根据这些信息来回答。首先，我可以思考一下这个PDF文档可能包含的内容：

1. **基础概念**：介绍什么是RAG、LlamaIndex等。
2. **技术实现细节**：如何使用LlamaDebugHandler进行调试和事件回溯。
3. **自定义Prompt的方法**：在中文环境下定制prompt template。

用户提供的示例查询是关于中医证候的，所以这个助手可能专注于医疗领域的应用。我可以基于这些信息来回答用户的问题。
</think>
根据你提供的文档内容，我们可以学到以下

准备文档目录：

创建一个名为my_documents的文件夹（或修改代码中的路径）

将你的PDF、Word和TXT文档放入该文件夹

运行程序：

技术解析

核心组件说明

文档加载器：使用PyPDF2和python-docx库处理不同格式的文档

嵌入模型：采用HuggingFace上的all-MiniLM-L6-v2模型（轻量级、开源）

向量数据库：FAISS提供高效的相似度搜索功能

问答链：使用RetrievalQA结合检索和生成

关键步骤分析

文档加载与预处理：

读取PDF文件内容（每页提取文本）

解析Word文档内容（合并所有段落）

向量化过程：

将文档内容转换为数字向量表示

使用FAISS建立索引，便于快速相似度搜索

查询处理流程：

用户输入问题 → 检索相关文档片段 → 生成回答

回答包含原始上下文信息，提高准确性

扩展建议

本地模型部署：可以使用Sentence Transformers库加载本地预训练模型

查询优化：添加纠错和意图识别功能提升用户体验

多模态支持：集成OCR技术处理图片文档（需要额外安装pytesseract）

知识更新机制：实现自动检测新文档并更新知识库

注意事项

该代码使用了开源嵌入模型，如果对性能要求高可以考虑付费模型

检索结果质量高度依赖于文档内容质量和数量的质量比

对于大型文档，建议先进行分块处理（chunking）提高检索效率

注意：如果你没有OpenAI API密钥，可以移除对OpenAI的依赖，改用其他开源LLM模型如GPT-J或使用本地语言模型（如HuggingFace上的DistilBERT问答模型）。同时，代码中包含了错误处理和示例文档生成功能，方便你测试系统。